In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

In [2]:
isamp = 3000
t = np.arange(0,1,0.01)
pps = 20
tsamp = pps*isamp

data_train = spi.loadmat('AD_5000_DP_TrData.mat')

u_in = data_train['u_in'][0:tsamp,:]
x_t_in = data_train['x_t_in'][0:tsamp,:]
s_in = data_train['s_in'][0:tsamp,:]

tsamp, t.shape, u_in.shape, x_t_in.shape, s_in.shape

(60000, (100,), (60000, 100), (60000, 1), (60000, 1))

In [3]:
max_u = np.max(u_in)
min_u = np.min(u_in)

u_in = (u_in-min_u)/(max_u-min_u)

max_t = np.max(x_t_in)
min_t = np.min(x_t_in)

x_t_in = (x_t_in-min_t)/(max_t-min_t)

max_s = np.max(s_in)
min_s = np.min(s_in)

s_in = (s_in-min_s)/(max_s-min_s)

max_s, min_s, max_u, min_u, max_t, min_t

(2.814752324700168,
 -3.0266588699038914,
 3.6565790163373184,
 -3.6177953494816775,
 0.99,
 0.0)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 10000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

hln = 30

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = Dense(hln, activation = "relu")(inputsB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (1,), name = 'inputsT')
hiddenT = Dense(hln, activation = "relu")(inputsT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = Dense(1)(combined)

model = Model(inputs = [inputsB, inputsT], outputs = output)    

model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001), loss = 'mse')
model.summary()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = './ChkPts/CI_VDN_s1_10000/',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

itr = 0
for i in range(0, 40):
    print('Iteration: '+str(itr))
    itr = itr+1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = 1,
              verbose = 1, batch_size = bs, callbacks = [model_checkpoint_callback])    

    itr_ps = 250-1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = itr_ps,
              verbose = 0, batch_size = bs, callbacks = [model_checkpoint_callback])
    itr = itr+itr_ps

print('Total iterations: '+str(itr))

model.load_weights('./ChkPts/CI_VDN_s1_10000/')

model.save_weights('./model/CI_VDN_s1_10000')

# model.load_weights('./model/Dense_model_TF_weights_CI_VDN')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           3030        ['inputsB[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 30)           60          ['inputsT[0][0]']                
                                                                                              

In [5]:
# data_test = spi.loadmat('test_ODE_AD.mat')

# u_in_test = data_test['X_test0']
# x_t_in_test = data_test['X_test1']
# s_in_test = data_test['y_test']

# u_in_test = (u_in_test-min_u)/(max_u-min_u)
# x_t_in_test = (x_t_in_test-min_t)/(max_t-min_t)
# s_in_test = (s_in_test-min_s)/(max_s-min_s)

# pred = model({"inputsB":u_in_test, "inputsT":x_t_in_test})

# print()
# print(np.mean((s_in_test-pred)**2))
# print(np.mean((s_in_test)**2))
# print(np.mean((s_in_test-pred)**2)/np.mean((s_in_test)**2))    

t = np.arange(0,1,0.01)
testdata = spi.loadmat('AD_TestData.mat')

u_in_test = testdata['u_in_test']
x_t_in_test = testdata['x_t_in_test']
s_in_test = testdata['s_in_test']

u_in_test.shape, x_t_in_test.shape, s_in_test.shape

u_in_test = (u_in_test-min_u)/(max_u-min_u)
x_t_in_test = (x_t_in_test-min_t)/(max_t-min_t)
s_in_test = (s_in_test-min_s)/(max_s-min_s)

pred = model({"inputsB":u_in_test, "inputsT":x_t_in_test}).numpy()

pred = (pred*(max_s-min_s))+min_s 
s_in_test = (s_in_test*(max_s-min_s))+min_s

print()
print(np.mean((s_in_test-pred)**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-pred)**2)/np.mean((s_in_test)**2))    


0.00011219960841149184
0.28067175942433253
0.00039975382148035524


In [6]:
# s_in_test.shape, pred.shape

In [7]:
# string = './CI_VDN_PDFs.mat'
# data_test = spi.savemat(string,{'pred':pred, 's_in_test':s_in_test})